In [ ]:
import numpy as np
import pandas as pd
import altair as alt
import holoviews as hv
import geoviews as gv
import param as pm
import panel as pn
from colorcet import cm
import datashader as ds
from holoviews.operation.datashader import rasterize, shade
from pyproj import Proj, transform

In [ ]:
# Need the Vega extension
pn.extension('vega')

In [ ]:
# Altair and Holoviews rendering in Jupyter notebook
alt.renderers.enable('default')
hv.extension('bokeh')

In [ ]:
# select necessary columns
usecols = ['n° horodateur','date horodateur','montant carte','durée payée (h)']
DATA = pd.read_csv("https://onedrive.live.com/download?cid=F1A01879C77A02B3&resid=F1A01879C77A02B3%21106&authkey=AKCxgVhALQhN_8c",sep=';',usecols=usecols) #, header=None, error_bad_lines=False)
# Trim data to avoid memory issues on Binder
N = 100000
DATA = DATA.sample(N)

We'll load the whole data and when the user changes parameters, we'll filter the full dataset in our app according to the parameters.

In [ ]:
# The colormaps available
cmaps = ['fire','kbc','bgy','bgyw','bmy','gray']
# Options for the basemap
opts = dict(xaxis=None, yaxis=None, bgcolor='ghostwhite', show_grid=False)
geodata = pd.read_csv('https://opendata.paris.fr/explore/dataset/horodateurs-mobiliers/download/?format=csv',sep=';')
horo = DATA.join(geodata.set_index(['numhoro']), on='n° horodateur', how='inner')
horo.regime = horo.geo_point_2d.apply(lambda x: str(x).split(',')[0]).astype(np.float32)
horo.geo_point_2d = horo.geo_point_2d.apply(lambda x: str(x).split(',')[1]).astype(np.float32)

In [ ]:
# transform the data to show on the map
horo.tarif = pd.to_datetime(horo["date horodateur"], utc=True).dt.hour
horo['geo_point_2d'], horo['regime'] = transform(Proj(init='epsg:4326'), Proj(init='epsg:3857'), horo['geo_point_2d'].tolist(), horo['regime'].tolist())

In [ ]:
class HoroApp(pm.Parameterized):
    """
    A Panel based dashboard app visualizing our data
    The app has three components:
        1. A datashaded heatmap
        2. Some widgets controlling the data plotted
        3. A bar chart of selected data
    The bar chart is linked to the Holoviews map and only plots the data currently visible on the map.
    """
    # Parameters
    alpha = pm.Magnitude(default=0.75, doc='Alpha value for opacity')
    cmap = pm.ObjectSelector(cm['fire'], objects={c: cm[c] for c in cmaps})
    hour = pm.Range(default=(0, 24), bounds=(0, 24))
    # Stream that gives the currently selected x_range/y_range of the map
    box = hv.streams.RangeXY(x_range=None, y_range=None)

    @pm.depends("hour")
    def points(self, x_range=None, y_range=None):
        """
        Get a Holoviews points object for the data. 
        Before returning filter the points by hour and x,y range.
        """
        # create the Points object holding all data
        points = hv.Points(horo, kdims=["geo_point_2d","regime"], vdims=["tarif"])

        # trim according to user inputs
        if self.hour != (0, 24):
            points = points.select(selection_expr=self.hour[0] <= hv.dim('tarif') <= self.hour[1])

        if x_range is not None:
            points = points.select(**{"geo_point_2d": x_range})

        if y_range is not None:
            points = points.select(**{"regime": y_range})

        return points

    def heatmap(self, **kwargs):
        """
        Return a datashaded heatmap of the data.
        """
        # create a dynamic map and link the box selection to it
        points = hv.DynamicMap(self.points, streams=[self.box])

        # aggregate the points by counting them
        aggregate = rasterize(points, x_sampling=1, y_sampling=1, width=800, height=400)

        tiles = gv.tile_sources.CartoDark().apply.opts(alpha=self.param.alpha, **opts)

        # datashaded heatmap
        heatmap = tiles * shade(aggregate, cmap=self.param.cmap)

        return heatmap.options(
            default_tools=['save', 'pan', 'box_zoom', 'reset'],
            active_tools=['box_zoom'],
            width=600,
            height=400,
        )

    @pm.depends("hour", "box.x_range", "box.y_range")
    def monthly_income(self):
        """
        Return a plot showing the mean income by month.
        This chart depends on the box selection's x,y range and will be 
        redrawn when the bounds are updated by the user.
        """
        months = ['January', 'February', 'March', 'April','May','June', 'July', 'August','September', 'October', 'November', 'December']
        # get the currently displayed points
        points = self.points(x_range=self.box.x_range, y_range=self.box.y_range)

        # Get the mean of income by months
        df = (points.data['tarifhor']*points.data['durée payée (h)']).groupby(
            pd.to_datetime(points.data['date horodateur'], utc=True).dt.month_name()).mean().rename_axis('Month').reindex(
            months).reset_index(name='Mean income')
        
        # return the Altair chart
        chart = (
            alt.Chart(df)
            .mark_bar()
            .encode(x=alt.X('Month:O', sort=alt.EncodingSortField(field='Month:O')), y='Mean income')
            .properties(width=500, height=300)
        )

        return pn.Pane(chart, width=800)

In [ ]:
# initialize our app
app = HoroApp(name="")


In [ ]:
# The app's title as an h2 element
title = pn.pane.HTML(
    '<h2>Visualizing with Datashader, Altair and Panel</h2>',
    style={'width': '800px', 'text-align': 'center'},
)
hist_title = pn.pane.HTML(
    '<h3>Income of the selected area by month</h3>',
    style={'width': '800px', 'text-align': 'center'},
)

In [ ]:
# Construct the dashboard
panel = pn.Column(
    pn.Row(title),
    pn.Row(pn.Param(app.param, expand_button=False, width=200), app.heatmap()),
    pn.Row(hist_title),
    pn.Row(pn.Spacer(width=75), app.monthly_income),
    align='center',
    width=1200,
)

### Call servable() to render our Panel app

This will:

1. Render the dashboard in the notebook
2. Enable the notebook to be served from `localhost`. (Execute `panel serve --show app.ipynb` from the command line and the app running live at `http://localhost:5006/app`)

In [ ]:
panel.servable()